In [11]:
from bing_image_downloader import downloader
import os
import csv
import shutil

In [20]:
# from bing_image_downloader import downloader

def download_images(request):

    # “””Store downloaded images in a directory.

    # Parameters:

    # request: list of queries to search for

        # Return: Images downloaded that match request

        # “””

    # print(request)

    downloader.download(request, 

                limit=10, 

                output_dir='images', 

                adult_filter_off=True, 

                force_replace=False, 

                timeout=60,
                
                filter = ['photo'],
                )



## Clean up functions, update files in csv after removing files

In [14]:
import re

def cleanup(file, files_name):
    folder_path = f'images/{file}'
    files = [f for f in os.listdir(folder_path) if not f.endswith('.csv')]
    files.sort()

    index = 1
    file_changes = []
    original_indexes = []
    new_indexes = []

    for filename in files:
        _, ext = os.path.splitext(filename)
        match = re.search(r'\d+', filename)  # Find the numeric part in the file name
        if match:
            file_index = int(match.group())  # Extract the index
            new_filename = f'{files_name}{index}{ext}'
            if filename != new_filename:
                file_changes.append([filename, new_filename])
            original_indexes.append(file_index)  # Append the original index
            new_indexes.append(index)
            os.rename(os.path.join(folder_path, filename), os.path.join(folder_path, new_filename))
            index += 1
    return original_indexes, new_indexes

def csv_update(file, original_indexes, new_indexes):
    # Path to the original CSV file
    original_csv_path = f'images/{file}/{file}.csv'

    # Path to the new CSV file
    new_csv_path = f'images/{file}/{file}_new.csv'

    # Read the original CSV file and extract links corresponding to original indexes
    links = {}
    with open(original_csv_path, mode='r') as csv_file:
        csv_reader = csv.DictReader(csv_file)
        for row in csv_reader:
            links[int(row['index'])] = row['link']

    # Create a new CSV file with new indexes and corresponding links
    with open(new_csv_path, mode='w', newline='') as new_csv_file:
        fieldnames = ['index', 'links']
        writer = csv.DictWriter(new_csv_file, fieldnames=fieldnames)
        writer.writeheader()
        for original_index, new_index in zip(original_indexes, new_indexes):
            writer.writerow({'index': new_index, 'links': links.get(original_index, '')})
def csv_status(file, original_indexes, new_indexes,new_csv_path):
     # Path to the original CSV file
    original_csv_path = f'images/{file}/{file}.csv'

    # Path to the new CSV file
    # new_csv_path = f'images/{file}/{file}_new.csv'
    # Path to the final CSV file
    final_csv_path = f'images/{file}/{file}_final.csv'

    # Read the original CSV file and extract links corresponding to original indexes
    links = {}
    with open(original_csv_path, mode='r') as csv_file:
        csv_reader = csv.DictReader(csv_file)
        for row in csv_reader:
            links[int(row['index'])] = row['link']

    # Create a copy of the new CSV file
    with open(final_csv_path, mode='w', newline='') as final_csv_file:
        fieldnames = ['index', 'links', 'status']
        writer = csv.DictWriter(final_csv_file, fieldnames=fieldnames)
        writer.writeheader()
        
        # Copy content from the new CSV file
        with open(new_csv_path, mode='r') as new_csv_file:
            csv_reader = csv.DictReader(new_csv_file)
            for row in csv_reader:
                writer.writerow({'index': row['index'], 'links': row['links'], 'status': 'used'})
        
        # Check for links from the original CSV file not present in the new CSV file
        for original_index, link in links.items():
            if original_index not in original_indexes:
                writer.writerow({'index': original_index, 'links': link, 'status': 'deleted'})

    # print(f"Final CSV file '{final_csv_path}' created successfully.")
def files_update(file, new_file_name):
    original_csv_path = f'images/{file}/{file}.csv'
    original_indexes = []
    new_indexes = []
    new_csv_path = f'images/{file}/{file}_new.csv'

    # Clean up the folder and update file names
    original_indexes,new_indexes  = cleanup(file, new_file_name)

    # Update the CSV file with new indexes
    csv_update(file, original_indexes,new_indexes)

    # Update the status in the final CSV file
    csv_status(file, original_indexes, new_indexes,new_csv_path)
def capitalize_and_underscore(names):
    renamed_names = []
    for name in names:
        # Split the name by space and capitalize each word
        capitalized_words = [word.capitalize() for word in name.split()]
        # Join the words with underscores
        renamed_name = '_'.join(capitalized_words)
        renamed_names.append(renamed_name)
    return renamed_names

## DBSCAN function to remove duplicates

In [15]:
import torch
import clip
from PIL import Image
import os
import numpy as np 
import pandas as pd
from collections import defaultdict
import matplotlib.pyplot as plt
from sklearn.cluster import DBSCAN



/opt/homebrew/anaconda3/lib/python3.11/site-packages/torchvision/io/image.py:13: UserWarning: Failed to load image Python extension: 'dlopen(/opt/homebrew/anaconda3/lib/python3.11/site-packages/torchvision/image.so, 0x0006): Symbol not found: __ZN3c1017RegisterOperatorsD1Ev
  Referenced from: <CFED5F8E-EC3F-36FD-AAA3-2C6C7F8D3DD9> /opt/homebrew/anaconda3/lib/python3.11/site-packages/torchvision/image.so
  Expected in:     <E459C462-F863-3A5A-AC9F-FD77B14BE845> /opt/homebrew/anaconda3/lib/python3.11/site-packages/torch/lib/libtorch_cpu.dylib'If you don't plan on using image functionality from `torchvision.io`, you can ignore this warning. Otherwise, there might be something wrong with your environment. Did you have `libjpeg` or `libpng` installed before building `torchvision` from source?
  warn(


In [54]:
def remove_duplicates(file_cluster_map): # function to remove duplicate files after labels made
    cluster_files = {}
    for file, label in file_cluster_map.items():
        if label != -1:  # Only consider non-unique one, should find only duplicate images
            if label not in cluster_files:
                cluster_files[label] = []
            cluster_files[label].append(file)

    # Delete all but one file from each non-noise cluster
    for label, files in cluster_files.items():
        # Keep the first file and delete the rest
        for file in files[1:]:
            if os.path.exists(file):
                os.remove(file)
                pass
            else:
                pass



## label device and clip preprocess method

In [16]:
device = "cuda" if torch.cuda.is_available() else "cpu"
model, preprocess = clip.load("ViT-B/32", device=device)

In [35]:
target_size = (256,256)
probs_end = []
logit_text = []
logit_image = []
embeddings = []

def create_embeddings(folder_path):
    direc = './images/US_politics/'
 
# target_size = (16,16)
   
    image_files = [os.path.join(direc, filename) for filename in os.listdir(direc) if filename.endswith('.jpg')]
    text = clip.tokenize(["Biden", "Trump", "Republican", "Democrat"]).to(device)
    return()

In [58]:
def create_embbedding_arrays(image_files):
    direc = './images/'+image_files
    embeddings = []
    target_size = (256,256)
    # target_size = (16,16)
    probs_end = []
    logit_text = []
    logit_image = []
    extensions = ('.jpg', '.png', '.img', '.jpeg')
    image_files = [os.path.join(direc, filename) for filename in os.listdir(direc) if filename.endswith(extensions)]

    # image_files = [os.path.join(direc, filename) for filename in os.listdir(direc) if filename.endswith('.jpg')]
    text = clip.tokenize(["Biden", "Trump", "Republican", "Democrat"]).to(device)
    
    for image_file in image_files:
        # Open and resize image
        with Image.open(image_file) as img:
            img_resized = img.resize(target_size)
        
        # Preprocess resized image
        image = preprocess(img_resized).unsqueeze(0).to(device)

        with torch.no_grad():
            image_features = model.encode_image(image)
            text_features = model.encode_text(text)
            
            logits_per_image, logits_per_text = model(image, text)
            probs = logits_per_image.softmax(dim=-1).numpy()  # Directly convert to NumPy array
        
        # Append the embedding to the list
        embeddings.append(image_features.cpu().numpy())
        logit_text.append(logits_per_text.cpu().numpy())
        logit_image.append(logits_per_image.cpu().numpy())
        probs_end.append(probs)

    # Convert embeddings list to a numpy array
    embeddings = np.array(embeddings)
    logit_text = np.array(logit_text)
    logit_image = np.array(logit_image)
    probs_end = np.array(probs_end)
    list_of_clustered_embeddings = [embeddings,logit_text,logit_image,probs_end]
    return list_of_clustered_embeddings, image_files

In [56]:
def cluster_labels(mapsfilter, image_files):
    embeddings_2d= mapsfilter[0]
    dbscan = DBSCAN(eps=0.01, min_samples=2, n_jobs=-1,metric="cosine")  # Compare images and label duplicates
    labels = dbscan.fit_predict(embeddings_2d)
    # Map each file to its corresponding cluster label
    file_cluster_map = {file: label for file, label in zip(image_files, labels)}
    remove_duplicates(file_cluster_map)


### Add types of photos you want to download to search array (ex: 'weapons' , 'kkk', 'trump') and it will create a new folder called 'weapons'/'kkk' and add images to it from bing.

# After curating and cleaning up the images you have downloaded run following to reorder files.

In [60]:
search_array = ['test']

for string in search_array:
    download_images(string)
    list_of_clustered_embeddings,image_files = create_embbedding_arrays(string)
    list_of_clustered_embeddings = [np.squeeze(item) for item in list_of_clustered_embeddings]
    cluster_labels(list_of_clustered_embeddings, image_files)
rename = capitalize_and_underscore(search_array)
for idx, nam in enumerate(search_array):

    files_update(nam,rename[idx])
    